In [2]:
import pandas as pd
from os.path import join


data_dir = r'C:\Users\fjn197\PhD\projects\PHAIR\pipelines\ehr_preprocess\ehr_preprocess\formatted_data\synthea5000'
diag = pd.read_csv(join(data_dir, 'concept.diagnose.csv'))
med = pd.read_csv(join(data_dir, 'concept.medication.csv'))

In [69]:
diag_full = pd.read_csv(r'C:\Users\fjn197\PhD\projects\PHAIR\pipelines\synthea\output\synthea5000\csv\conditions.csv')

In [86]:
def search_by_name(df, name):
    mask = df.DESCRIPTION.str.lower().str.contains(name.lower())
    # get a dictionary of unique descriptions to their codes
    masked_df= df.loc[mask, ['DESCRIPTION', 'CODE']].drop_duplicates()
    return dict(zip(masked_df.DESCRIPTION, masked_df.CODE))
    

In [105]:
heart_disease = search_by_name(diag_full, 'heart')
stroke = search_by_name(diag_full, 'stroke')
infarction = search_by_name(diag_full, 'infarction')
tvr = search_by_name(diag_full, 'revascularization')
print(tvr)

{}


## Examine MACE outcomes

In [101]:
mace_codes = [22298006, 230690007,  4557003, 84114007, 414545008,
              230690007, 401303003, 401314000, 22298006
              
              ]
mace_codes = ['D' + str(x) for x in mace_codes]
mace_pids = []
for mace_code in mace_codes:
    print(mace_code, end=' ')
    mace_df = diag[diag['CONCEPT'] == mace_code]
    print('number of patients', mace_df.PID.nunique(), end=' ')
    print('number of events', mace_df.shape[0],)
    mace_pids.extend(mace_df.PID.unique())

D22298006 number of patients 143 number of events 143
D230690007 number of patients 39 number of events 39
D4557003 number of patients 13 number of events 13
D84114007 number of patients 8 number of events 8
D414545008 number of patients 1097 number of events 1097
D230690007 number of patients 39 number of events 39
D401303003 number of patients 139 number of events 139
D401314000 number of patients 152 number of events 152
D22298006 number of patients 143 number of events 143


In [59]:
diag.CONCEPT.value_counts()[25:40]

D237602007         1172
D274531002         1104
D414545008         1097
D80583007          1010
D19169002          1004
D278860009         1002
D127013003         1000
D431855005          960
D25675004           902
D125605004          879
D312608009          876
D43878008           875
D90781000119102     873
D65363002           856
D384709000          844
Name: CONCEPT, dtype: int64

## Examine statin codes

In [18]:
med

,TIMESTAMP,PID,ADMISSION_ID,CONCEPT
0,1914-06-14T14:58:43Z,e7aec45f-9052-e274-ce16-2b9cb676e533,298ab82a-dc57-6ea2-5ce8-39bf30f67b37,M243670
1,1915-05-11T18:48:54Z,92f53d3d-aacf-6726-56bd-b76227f88357,d5554bf0-6416-af56-431f-aafdd3922922,M243670
2,1915-08-20T02:06:38Z,a423cc04-f042-111e-fc70-6570cc3122fb,76e19af4-deb9-a920-e377-40797d0083f9,M243670
3,1915-11-24T08:40:05Z,e394980d-e61b-df87-27a5-73894358c04b,93d05ad6-76de-7185-d877-ef86d450488e,M243670
4,1916-05-01T02:06:38Z,a423cc04-f042-111e-fc70-6570cc3122fb,803b0247-5d08-89f7-07f7-b716df21c880,M243670
...,...,...,...,...
646975,2024-06-27T03:35:55Z,38b6be96-d490-da03-db7a-599feaa4b184,e0a5ae38-5459-c213-6481-336c36867f3b,M308136
646976,2024-06-27T04:21:24Z,ac4afdcb-132d-6f59-8ea1-a2c896af50bd,44e00872-d7c8-ac77-2284-d7de2fdb155e,M308136
646977,2024-06-27T04:21:24Z,ac4afdcb-132d-6f59-8ea1-a2c896af50bd,44e00872-d7c8-ac77-2284-d7de2fdb155e,M106892
646978,2024-06-27T04:21:24Z,ac4afdcb-132d-6f59-8ea1-a2c896af50bd,44e00872-d7c8-ac77-2284-d7de2fdb155e,M856987


In [31]:
statin_codes = [617312, 312961,312962, 19821, 7597, 34482]
statin_codes = ['M' + str(x) for x in statin_codes]
for statin_code in statin_codes:
    print(statin_code)
    print('number of patients', med[med['CONCEPT'].str.startswith(statin_code)].PID.nunique())
    print('number of events', med[med['CONCEPT'].str.startswith(statin_code)].shape[0])

M617312
number of patients 42
number of events 42
M312961
number of patients 897
number of events 897
M312962
number of patients 1
number of events 1
M19821
number of patients 23
number of events 23
M7597
number of patients 0
number of events 0
M34482
number of patients 0
number of events 0


We will take M312961 (Simvastatin) as an example since that is quite often used.

In [38]:
simvastatin_codes = [312961, 312962, 1189803, 484211, 1430892, 803516]
simvastatin_codes = ['M' + str(x) for x in simvastatin_codes]
simvastatin_pids = []
for statin_code in simvastatin_codes:
    print(statin_code)
    simvastatin_df = med[med['CONCEPT']==statin_code]
    print('number of patients', simvastatin_df.PID.nunique())
    print('number of events', simvastatin_df.shape[0])
    simvastatin_pids.extend(simvastatin_df.PID.unique())

M312961
number of patients 897
number of events 897
M312962
number of patients 1
number of events 1
M1189803
number of patients 0
number of events 0
M484211
number of patients 0
number of events 0
M1430892
number of patients 0
number of events 0
M803516
number of patients 0
number of events 0


In [39]:
print('number of patients with MACE', len(set(mace_pids)))
print('number of patients with simvastatin', len(set(simvastatin_pids)))
print('number of patients with both', len(set(mace_pids).intersection(set(simvastatin_pids))) )

number of patients with MACE 198
number of patients with simvastatin 897
number of patients with both 88
